## Word Generation

***

### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import string

from random import randint


import nltk
from nltk.tokenize import word_tokenize


%matplotlib inline
sns.set_style('darkgrid')
sns.set(font_scale=1.2)
#sets the default autosave frequency in seconds
%autosave 60

# from sklearn.neural_network import MLPClassifier, MLPRegressor
# from sklearn.utils import shuffle

#from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV, RandomizedSearchCV,StratifiedKFold
#from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, OneHotEncoder
#from sklearn.metrics import confusion_matrix, classification_report, mean_absolute_error, mean_squared_error,r2_score
#from sklearn.metrics import plot_confusion_matrix, plot_precision_recall_curve, plot_roc_curve, accuracy_score
#from sklearn.metrics import auc, f1_score, precision_score, recall_score, roc_auc_score

import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras.models import Sequential, save_model
# from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation
# from tensorflow.keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
# from tensorflow.keras.optimizers import Adam

#For RNN
import keras
from keras.models import save_model, load_model, Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten, LSTM, Embedding
from keras.optimizers import Adam, Adagrad, RMSprop, SGD
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras.regularizers import l1,l2,l1_l2
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical, get_file
from keras.preprocessing.sequence import pad_sequences


#For Tuning
from keras.callbacks import EarlyStopping, LearningRateScheduler, ModelCheckpoint, ReduceLROnPlateau, TensorBoard

#from imblearn.under_sampling import RandomUnderSampler
#from imblearn.over_sampling import RandomOverSampler

import warnings
warnings.filterwarnings('ignore')

import pickle
from pickle import dump, load

pd.set_option('display.max_columns',None)
#pd.set_option('display.max_rows',None)
pd.set_option('display.width', 1000)
pd.option_context('float_format', '{:.2f}'.format)

np.random.seed(0)
np.set_printoptions(suppress=True)
tf.random.set_seed(0)

Autosaving every 60 seconds


## File Loading

### 1st Method

In [2]:
#f = open("textfile2.txt",'r', encoding='utf-8').read()  

In [3]:
#f;

In [4]:
#text = str(f)

In [5]:
#text

In [6]:
#text = text.splitlines()

In [7]:
#len(text)

In [8]:
#text

### 2nd Method

In [9]:
#file = open("textfile2.txt", 'rb').read().decode('utf-8')

In [10]:
#file

In [11]:
#type(file)

In [12]:
#len(set(file))

### 3rd Method

In [13]:
df = pd.read_csv("blogdata_1000.csv")

In [14]:
df.head()

,words
0,"We have given our hearts away, a sordid boon !”"
1,1. Start it on the side
2,"Sugar’s sweet, so is she,"
3,So because the Asian community was so by need ...
4,Nicholas’ brain runs on and is excellent at pr...


In [15]:
df.shape

(1000, 1)

### Tokenization inside data frame

In [16]:
df['tokenized'] = df['words'].apply(word_tokenize)

In [17]:
df.head()

,words,tokenized
0,"We have given our hearts away, a sordid boon !”","[We, have, given, our, hearts, away, ,, a, sor..."
1,1. Start it on the side,"[1, ., Start, it, on, the, side]"
2,"Sugar’s sweet, so is she,","[Sugar, ’, s, sweet, ,, so, is, she, ,]"
3,So because the Asian community was so by need ...,"[So, because, the, Asian, community, was, so, ..."
4,Nicholas’ brain runs on and is excellent at pr...,"[Nicholas, ’, brain, runs, on, and, is, excell..."


In [18]:
punc = string.punctuation

In [19]:
df['no_punc'] = df['tokenized'].apply(lambda x: [word for word in x if word not in punc])

In [20]:
df.head()

,words,tokenized,no_punc
0,"We have given our hearts away, a sordid boon !”","[We, have, given, our, hearts, away, ,, a, sor...","[We, have, given, our, hearts, away, a, sordid..."
1,1. Start it on the side,"[1, ., Start, it, on, the, side]","[1, Start, it, on, the, side]"
2,"Sugar’s sweet, so is she,","[Sugar, ’, s, sweet, ,, so, is, she, ,]","[Sugar, ’, s, sweet, so, is, she]"
3,So because the Asian community was so by need ...,"[So, because, the, Asian, community, was, so, ...","[So, because, the, Asian, community, was, so, ..."
4,Nicholas’ brain runs on and is excellent at pr...,"[Nicholas, ’, brain, runs, on, and, is, excell...","[Nicholas, ’, brain, runs, on, and, is, excell..."


In [21]:
df['final'] = df['no_punc'].apply(lambda x: [word.lower() for word in x if word.isalpha()])

In [22]:
df.head()

,words,tokenized,no_punc,final
0,"We have given our hearts away, a sordid boon !”","[We, have, given, our, hearts, away, ,, a, sor...","[We, have, given, our, hearts, away, a, sordid...","[we, have, given, our, hearts, away, a, sordid..."
1,1. Start it on the side,"[1, ., Start, it, on, the, side]","[1, Start, it, on, the, side]","[start, it, on, the, side]"
2,"Sugar’s sweet, so is she,","[Sugar, ’, s, sweet, ,, so, is, she, ,]","[Sugar, ’, s, sweet, so, is, she]","[sugar, s, sweet, so, is, she]"
3,So because the Asian community was so by need ...,"[So, because, the, Asian, community, was, so, ...","[So, because, the, Asian, community, was, so, ...","[so, because, the, asian, community, was, so, ..."
4,Nicholas’ brain runs on and is excellent at pr...,"[Nicholas, ’, brain, runs, on, and, is, excell...","[Nicholas, ’, brain, runs, on, and, is, excell...","[nicholas, brain, runs, on, and, is, excellent..."


In [23]:
words = df['final']
allwords = []
for wordlist in words:
    allwords+= wordlist

In [24]:
allwords

['we',
 'have',
 'given',
 'our',
 'hearts',
 'away',
 'a',
 'sordid',
 'boon',
 'start',
 'it',
 'on',
 'the',
 'side',
 'sugar',
 's',
 'sweet',
 'so',
 'is',
 'she',
 'so',
 'because',
 'the',
 'asian',
 'community',
 'was',
 'so',
 'by',
 'need',
 'the',
 'next',
 'thing',
 'you',
 'know',
 'i',
 'have',
 'asian',
 'kids',
 'following',
 'me',
 'around',
 'you',
 'know',
 'doing',
 'more',
 'shit',
 'for',
 'me',
 'than',
 'the',
 'boys',
 'that',
 'wanted',
 'inside',
 'of',
 'me',
 'especially',
 'the',
 'two',
 'in',
 'particular',
 'they',
 'thought',
 'i',
 'was',
 'so',
 'wonderful',
 'which',
 'sickened',
 'me',
 'even',
 'further',
 'i',
 'was',
 'not',
 'wonderful',
 'it',
 'reminds',
 'me',
 'of',
 'that',
 'quote',
 'from',
 'wwii',
 'era',
 'forget',
 'it',
 'but',
 'goes',
 'on',
 'to',
 'say',
 'something',
 'like',
 'it',
 'was',
 'not',
 'that',
 'i',
 'was',
 'a',
 'hero',
 'it',
 'was',
 'everyone',
 'around',
 'me',
 'was',
 'acting',
 'so',
 'badly',
 'when',
 '

In [25]:
len(allwords)

39018

In [26]:
type(allwords)

list

### Create Sequences of Tokens

In [27]:
# organize into sequences of tokens
train_len = 25+1 # 50 training words , then one target word

# Empty list of sequences
text_sequences = []

for i in range(train_len, len(allwords)):
    
    # Grab train_len# amount of characters
    seq = allwords[i-train_len:i]
    
    # Add to list of sequences
    text_sequences.append(seq)

In [28]:
' '.join(text_sequences[0])

'we have given our hearts away a sordid boon start it on the side sugar s sweet so is she so because the asian community was'

In [29]:
' '.join(text_sequences[1])

'have given our hearts away a sordid boon start it on the side sugar s sweet so is she so because the asian community was so'

In [30]:
' '.join(text_sequences[2])

'given our hearts away a sordid boon start it on the side sugar s sweet so is she so because the asian community was so by'

In [31]:
len(text_sequences)

38992

###  Keras Tokenization

In [32]:
token = Tokenizer()

In [33]:
token.fit_on_texts(text_sequences) #Only the created sequences

In [34]:
sequences = token.texts_to_sequences(text_sequences)

In [35]:
sequences

[[21,
  19,
  268,
  56,
  3005,
  154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15],
 [19,
  268,
  56,
  3005,
  154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15,
  25],
 [268,
  56,
  3005,
  154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15,
  25,
  36],
 [56,
  3005,
  154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15,
  25,
  36,
  123],
 [3005,
  154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15,
  25,
  36,
  123,
  1],
 [154,
  4,
  7479,
  7478,
  209,
  8,
  14,
  1,
  301,
  981,
  23,
  327,
  25,
  9,
  38,
  25,
  75,
  1,
  1279,
  980,
  15,
  25,
  36,
  123,
  1,
  155],
 

In [36]:
sequences[0]

[21,
 19,
 268,
 56,
 3005,
 154,
 4,
 7479,
 7478,
 209,
 8,
 14,
 1,
 301,
 981,
 23,
 327,
 25,
 9,
 38,
 25,
 75,
 1,
 1279,
 980,
 15]

In [37]:
sequences[1]

[19,
 268,
 56,
 3005,
 154,
 4,
 7479,
 7478,
 209,
 8,
 14,
 1,
 301,
 981,
 23,
 327,
 25,
 9,
 38,
 25,
 75,
 1,
 1279,
 980,
 15,
 25]

In [38]:
token.word_counts

OrderedDict([('we', 5799),
             ('have', 5878),
             ('given', 393),
             ('our', 2526),
             ('hearts', 31),
             ('away', 708),
             ('a', 25097),
             ('sordid', 8),
             ('boon', 9),
             ('start', 530),
             ('it', 13531),
             ('on', 8231),
             ('the', 49930),
             ('side', 352),
             ('sugar', 119),
             ('s', 5268),
             ('sweet', 329),
             ('so', 4823),
             ('is', 12967),
             ('she', 3244),
             ('because', 1582),
             ('asian', 102),
             ('community', 129),
             ('was', 7462),
             ('by', 3328),
             ('need', 910),
             ('next', 702),
             ('thing', 572),
             ('you', 9256),
             ('know', 1950),
             ('i', 23660),
             ('kids', 364),
             ('following', 286),
             ('me', 4134),
             ('around', 988),
     

In [39]:
token.word_index

{'the': 1,
 'and': 2,
 'to': 3,
 'a': 4,
 'i': 5,
 'of': 6,
 'in': 7,
 'it': 8,
 'is': 9,
 'that': 10,
 'for': 11,
 'you': 12,
 'with': 13,
 'on': 14,
 'was': 15,
 'my': 16,
 'this': 17,
 'as': 18,
 'have': 19,
 'but': 20,
 'we': 21,
 'are': 22,
 's': 23,
 'be': 24,
 'so': 25,
 'he': 26,
 'me': 27,
 'they': 28,
 'at': 29,
 'from': 30,
 'his': 31,
 'not': 32,
 'or': 33,
 'one': 34,
 'all': 35,
 'by': 36,
 'up': 37,
 'she': 38,
 'your': 39,
 'her': 40,
 'some': 41,
 'an': 42,
 'when': 43,
 'just': 44,
 'can': 45,
 'do': 46,
 'like': 47,
 'their': 48,
 'what': 49,
 'out': 50,
 'time': 51,
 'who': 52,
 'about': 53,
 'will': 54,
 'there': 55,
 'our': 56,
 'if': 57,
 'more': 58,
 'had': 59,
 't': 60,
 'them': 61,
 'which': 62,
 'were': 63,
 'how': 64,
 'would': 65,
 'been': 66,
 'know': 67,
 'has': 68,
 'into': 69,
 'him': 70,
 'here': 71,
 'get': 72,
 'people': 73,
 'also': 74,
 'because': 75,
 'first': 76,
 'other': 77,
 'day': 78,
 'then': 79,
 'much': 80,
 'these': 81,
 'now': 82,
 'coul

In [40]:
x = ['i love you'] #Test the tokens

In [41]:
token.texts_to_sequences(x)

[[5, 118, 12]]

In [42]:
vocab_size = len(token.word_counts)

In [43]:
vocab_size

7479

### Convert to Numpy Matrix

In [44]:
sequences = np.array(sequences)

In [45]:
sequences

array([[  21,   19,  268, ..., 1279,  980,   15],
       [  19,  268,   56, ...,  980,   15,   25],
       [ 268,   56, 3005, ...,   15,   25,   36],
       ...,
       [  22,   71,   75, ...,    2,  449,   14],
       [  71,   75,    6, ...,  449,   14,   13],
       [  75,    6,   10, ...,   14,   13,   16]])

## Train-Test Split

In [46]:
sequences.shape

(38992, 26)

In [47]:
# First 49 words
sequences[:,:-1]

array([[  21,   19,  268, ...,    1, 1279,  980],
       [  19,  268,   56, ..., 1279,  980,   15],
       [ 268,   56, 3005, ...,  980,   15,   25],
       ...,
       [  22,   71,   75, ..., 3004,    2,  449],
       [  71,   75,    6, ...,    2,  449,   14],
       [  75,    6,   10, ...,  449,   14,   13]])

In [48]:
# last Word
sequences[:,-1]

array([15, 25, 36, ..., 14, 13, 16])

In [49]:
X = sequences[:,:-1]

In [50]:
y = sequences[:,-1]

In [51]:
y = to_categorical(y, num_classes=vocab_size+1)

In [52]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [53]:
seq_len = X.shape[1]

In [54]:
seq_len

25

## Model Training

### Creating an LSTM based model

In [55]:
model = Sequential()
model.add(Embedding(vocab_size+1, 25, input_length=seq_len))
model.add(LSTM(50, return_sequences=True))
model.add(LSTM(50))
model.add(Dense(50, activation='relu'))
model.add(Dense(vocab_size+1, activation="softmax"))

In [56]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 25, 25)            187000    
_________________________________________________________________
lstm (LSTM)                  (None, 25, 50)            15200     
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                20200     
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dense_1 (Dense)              (None, 7480)              381480    
Total params: 606,430
Trainable params: 606,430
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [58]:
model.fit(X,y, batch_size=128, epochs=10, verbose=1)

Epoch 1/10
305/305 [==============================] - 23s 62ms/step - loss: 7.6938 - accuracy: 0.0376
Epoch 2/10
305/305 [==============================] - 19s 62ms/step - loss: 6.8502 - accuracy: 0.0488
Epoch 3/10
305/305 [==============================] - 19s 63ms/step - loss: 6.7278 - accuracy: 0.0500
Epoch 4/10
305/305 [==============================] - 22s 71ms/step - loss: 6.6072 - accuracy: 0.0521
Epoch 5/10
305/305 [==============================] - 23s 76ms/step - loss: 6.4913 - accuracy: 0.0612
Epoch 6/10
305/305 [==============================] - 25s 83ms/step - loss: 6.3974 - accuracy: 0.0640
Epoch 7/10
305/305 [==============================] - 22s 72ms/step - loss: 6.3298 - accuracy: 0.0679
Epoch 8/10
305/305 [==============================] - 20s 65ms/step - loss: 6.2671 - accuracy: 0.0701
Epoch 9/10
305/305 [==============================] - 20s 67ms/step - loss: 6.2107 - accuracy: 0.0734
Epoch 10/10
305/305 [==============================] - 19s 64ms/step - loss: 6.107

### Save the Model

In [59]:
model.save("wordgen.h5")

In [60]:
dump(token, open('my_simpletokenizer','wb'))

### Generating New Text - Predict one next word

In [61]:
# Final Output
output_text = []
    
# Initial Seed Sequence
input_text = "This is a text"
    
# Create num_gen_words
for i in range(0,1):
        
    # Take the input text string and encode it to a sequence
    encoded_text = token.texts_to_sequences([input_text])[0]
        
    # Pad sequences to our trained rate (50 words in the video)
    pad_encoded = pad_sequences([encoded_text], maxlen=seq_len, truncating='pre')
        
    # Predict Class Probabilities for each word
    pred_word_ind = model.predict_classes(pad_encoded, verbose=0)[0]
        
    # Grab word
    pred_word = token.index_word[pred_word_ind] 
    
    # Update the sequence of input text (shifting one over with the new word)
    input_text += ' ' + pred_word
        
    output_text.append(pred_word)

In [62]:
output_text

['and']

#### Python code done by Dennis Lam